In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
import joblib
import winsound

In [ ]:
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)
X = mnist["data"]
y = mnist["target"].astype(np.uint8)

In [ ]:
#Först delar vi upp datan i träning och test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Normalisera pixelvärdena (skala till intervall [0, 1]) vilket gör att alla egenskaper får samma vikt, vilket vissa modeller är känslig för.
X_train_normalized = X_train / 255.0
X_test_normalized = X_test / 255.0

#Eftersom vi har träningsdatan på 56000, vill jag snabba upp finjustering av hyperparameter genom att dela upp denna data i hälften de första två omgångarna för finjustering.
X_train_partial, X_part, y_train_partial, y_part = train_test_split(X_train_normalized, y_train, test_size=0.5, random_state=42)

Börjar med att först göra test på hälften av träningsdatan

RFC 10, 50, 100 på n__estimators, none, 10 och 20 på max depth.
Den fick 100 och none som bäst hyperparametrar. Blev ett score påÄndrar dem till 80, 100, 120 i omgång två. Ändrar depth till 30, 50, 80.

ETC 100, 200, 500 n_estimators, max depth none, 10 och 20, min samples split 2, 5 och 10, min samples leaf 1, 2 och 4.
Bästa parametrar: {'ETC__max_depth': None, 'ETC__min_samples_leaf': 1, 'ETC__min_samples_split': 2, 'ETC__n_estimators': 500}
KNN neighbors 3, 5 och 7, weights uniform och distance.

Resultat första omgång med halva träningsset med följande hyperparametrar
'RFC': (RandomForestClassifier(random_state=42), {
        'RFC__n_estimators': [10, 50, 100],
        'RFC__max_depth': [10, 20, 50]
    })
    ,'ETC': (ExtraTreesClassifier(random_state=42, bootstrap=True), {
        'ETC__n_estimators': [100, 200, 500],
        'ETC__max_depth': [10, 20, 50],
        'ETC__min_samples_split': [2, 5, 10],
        'ETC__min_samples_leaf': [1, 2, 4]
    })
    ,'KNN': (KNeighborsClassifier(), {'KNN__n_neighbors': [3, 5, 7], 'KNN__weights': ['uniform', 'distance']}) Jag kommer inte ändra KNN för hela träningsset.


    Utför GridSearchCV för RFC...
För bästa RFC modell: 
Bästa parametrar: {'RFC__max_depth': 20, 'RFC__n_estimators': 100}
Träningsnoggrannhet med bästa parametrar: 0.9584
Utför GridSearchCV för ETC...
För bästa ETC modell: 
Bästa parametrar: {'ETC__max_depth': 50, 'ETC__min_samples_leaf': 1, 'ETC__min_samples_split': 2, 'ETC__n_estimators': 500}
Träningsnoggrannhet med bästa parametrar: 0.9608
Utför GridSearchCV för KNN...
För bästa KNN modell: 
Bästa parametrar: {'KNN__n_neighbors': 3, 'KNN__weights': 'distance'}
Träningsnoggrannhet med bästa parametrar: 0.9600

Bästa modellen är: Pipeline(steps=[('ETC',
                 ExtraTreesClassifier(bootstrap=True, max_depth=50,
                                      n_estimators=500, random_state=42))])
Bästa träningsnoggrannhet: 0.9608


Andra omgången med halva träningsdatan och följande hyperparametrar
'RFC': (RandomForestClassifier(random_state=42), {
        'RFC__n_estimators': [80, 100, 120],
        'RFC__max_depth': [15, 20, 30]
    })
    ,'ETC': (ExtraTreesClassifier(random_state=42, bootstrap=True), {
        'ETC__n_estimators': [500, 600],
        'ETC__max_depth': [50, 75, 100],
        'ETC__min_samples_split': [2],
        'ETC__min_samples_leaf': [1]
    })
    ,'KNN': (KNeighborsClassifier(), {'KNN__n_neighbors': [3, 5, 7, 10], 'KNN__weights': ['uniform', 'distance']})
Utför GridSearchCV för RFC...
För bästa RFC modell: 
Bästa parametrar: {'RFC__max_depth': 30, 'RFC__n_estimators': 120}
Träningsnoggrannhet med bästa parametrar: 0.9589
Utför GridSearchCV för ETC...
För bästa ETC modell: 
Bästa parametrar: {'ETC__max_depth': 50, 'ETC__min_samples_leaf': 1, 'ETC__min_samples_split': 2, 'ETC__n_estimators': 500}
Träningsnoggrannhet med bästa parametrar: 0.9608
Utför GridSearchCV för KNN...
För bästa KNN modell: 
Bästa parametrar: {'KNN__n_neighbors': 3, 'KNN__weights': 'distance'}
Träningsnoggrannhet med bästa parametrar: 0.9600

Bästa modellen är: Pipeline(steps=[('ETC',
                 ExtraTreesClassifier(bootstrap=True, max_depth=50,
                                      n_estimators=500, random_state=42))])
Bästa träningsnoggrannhet: 0.9608

In [ ]:
# Skapar en dictionary med modellerna vi vill använda.
models = {
   'RFC': (RandomForestClassifier(random_state=42), {
        'RFC__n_estimators': [500],
        'RFC__max_depth': [15],
        'RFC__min_samples_split': [10],
        'RFC__min_samples_leaf': [5]
    }),
    'ETC': (ExtraTreesClassifier(random_state=42, bootstrap=True), {
        'ETC__n_estimators': [500],
        'ETC__max_depth': [15],
        'ETC__min_samples_split': [10],
        'ETC__min_samples_leaf': [5]
    }),
    'KNN': (KNeighborsClassifier(), {'KNN__n_neighbors': [5, 14, 25], 'KNN__weights': ['uniform', 'distance']})
}

# Vi initierar best_accuracy och best_model där vi kommer spara uppdatera i vår for loop.
best_accuracy = 0
best_model = None

In [ ]:
# Kör GridSearchCV för varje modell och alla hyperparametrar vi initierat i vår models dictionary.
for model_name, (model, params) in models.items():
    
    pipeline = Pipeline([
        (model_name, model) 
    ]) # Bygger en pipeline för modellerna
    
    # GridsearchCV delen, där vi kör hyperparameteroptimering med 5 folds och använder alla processorkärnor för att maximera resurs för träning och minska tid.
    print(f"Utför GridSearchCV för {model_name}...")

    grid_search = GridSearchCV(pipeline, params, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train_normalized, y_train)  # Vi fittar på den normaliserad datan, här har vi vid tidigare träningar haft den partiella.
    
    #Här lägger vi den bästa modelen med parametrar och score.
    best_model_for_this = grid_search.best_estimator_
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    
    print(f"För bästa {model_name} modell: \nBästa parametrar: {best_params}")
    print(f"Träningsnoggrannhet med bästa parametrar: {best_score:.4f}")
    
    # Här ersätts efter varje iteration best_accuracy och best_model med den som har för närvarande bäst
    if best_score > best_accuracy:
        best_accuracy = best_score
        best_model = best_model_for_this
    winsound.Beep(2000,500) #Frekvens: 2000 Hz, varaktighet: 500ms för att ge en audio cue när den är klar med en modell
winsound.Beep(800,700) #Frekvens: 800 Hz, varaktighet: 700ms för en tydlig audio cue när hela träningen är slut.
# Sammanfattning av bästa modellen
print(f"\nBästa modellen är: {best_model}")
print(f"Bästa träningsnoggrannhet: {best_accuracy:.4f}")

Utför GridSearchCV för RFC...
För bästa RFC modell: 
Bästa parametrar: {'RFC__max_depth': 15, 'RFC__min_samples_leaf': 5, 'RFC__min_samples_split': 10, 'RFC__n_estimators': 500}
Träningsnoggrannhet med bästa parametrar: 0.9605
Utför GridSearchCV för ETC...
För bästa ETC modell: 
Bästa parametrar: {'ETC__max_depth': 15, 'ETC__min_samples_leaf': 5, 'ETC__min_samples_split': 10, 'ETC__n_estimators': 500}
Träningsnoggrannhet med bästa parametrar: 0.9557
Utför GridSearchCV för KNN...
För bästa KNN modell: 
Bästa parametrar: {'KNN__n_neighbors': 5, 'KNN__weights': 'distance'}
Träningsnoggrannhet med bästa parametrar: 0.9701

Bästa modellen är: Pipeline(steps=[('KNN', KNeighborsClassifier(weights='distance'))])
Bästa träningsnoggrannhet: 0.9701


In [ ]:
"""# Spara den bästa modellen till en fil
joblib.dump(best_model, 'best_model.pkl')

print("Modellen har sparats som 'best_model.pkl'")"""
#Har här kommenterat ut koden för att inte råka spara över min sparade modell.

Modellen har sparats som 'best_model.pkl'
